<a href="https://colab.research.google.com/github/jugalpanchal/bd-chef/blob/main/spark_cheat_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Follow the steps to install the dependencies:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # install java
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz # spark package download
!tar xf spark-3.1.2-bin-hadoop3.2.tgz # unzip spark package
!pip install -q findspark # install spark

# Set the location of Java and Spark:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# create or get spark session
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Spark_App1") \
        .getOrCreate()

sc = spark.sparkContext

##Join

In [4]:
# default inner
# inner, cross, outer, full, fullouter, full_outer, 
# left, leftouter, left_outer, 
# right, rightouter, right_outer, 
# semi, leftsemi, left_semi, 
# anti, leftanti, left_anti

In [14]:
df1 = spark.createDataFrame([('TM1', 'Jugal', 'T101'), ('TM2', 'Garvik', 'T102'), ('TM3', 'John', 'T103'), ('TM4', 'Mike', 'T102')]) # pass some tuples within a collection
df1 = df1.toDF('tm_id', 'tm_name', 'tm_team_id')

df2 = spark.createDataFrame([('T101', 'Sun'), ('T102', 'Moon'), ('T103', 'Mars')])
df2 = df2.toDF('team_id', 'team_name')

df1.show()
df2.show()

+-----+-------+----------+
|tm_id|tm_name|tm_team_id|
+-----+-------+----------+
|  TM1|  Jugal|      T101|
|  TM2| Garvik|      T102|
|  TM3|   John|      T103|
|  TM4|   Mike|      T102|
+-----+-------+----------+

+-------+---------+
|team_id|team_name|
+-------+---------+
|   T101|      Sun|
|   T102|     Moon|
|   T103|     Mars|
+-------+---------+



In [15]:
# join
